In [2]:
from Chemometrics.import_common import *
from Chemometrics.mass_utils import iso_dist
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'

In [3]:
import os
datafile = 'data050.mzXML'

In [50]:
from pyteomics import mzxml, auxiliary, mass

In [54]:
mass.Composition('C2H1O') + mass.Composition('C3H1O')*2

Composition({'C': 8, 'H': 3, 'O': 3})

In [5]:
def updateROIs(ROIs, spectrum, intensity_threshold=500, _threshold=.0001):
    """
    centwave
    """
    if not ROIs:
        ROIs = {'m/z': spectrum['m/z array'].copy(), 
                'intensity': spectrum['intensity array'].copy()}
        return ROIs
    contin_ = [False for mz in ROIs['m/z']] #todo
    new_mz =[True for mz in spectrum['m/z array']]
    for mz_id in range(len(ROIs['m/z'])):
        for _mz1_id in range(len(spectrum['m/z array'])):
            _mz = ROIs['m/z'][mz_id]
            _int = ROIs['intensity'][mz_id]
            _mz1 = spectrum['m/z array'][_mz1_id]
            _int1 = spectrum['intensity array'][_mz1_id]
            if (abs(_mz - _mz1) < _threshold) and new_mz[_mz1_id]:
                intens_ = _int1 + _int
                ROIs['intensity'][mz_id] = intens_
                w = _int1/intens_
                ROIs['m/z'][mz_id] = w*_int1 + (1-w)*_int
                contin_[mz_id] = True
                new_mz[_mz1_id] = False
    _strong = ROIs['intensity'] > intensity_threshold
    _keep = np.logical_or(contin_, _strong)
    ROIs['m/z'] = np.hstack([ROIs['m/z'][_keep], spectrum['m/z array'][new_mz]])
    ROIs['intensity'] = np.hstack([ROIs['intensity'][_keep], spectrum['intensity array'][new_mz]])
    return ROIs

In [49]:
%%time
ROIs = {}
with mzxml.read(datafile) as reader:
    for spectrum in reader:
        if spectrum['msLevel'] == 1:
            ROIs = updateROIs(ROIs, spectrum)

Wall time: 21min 15s
